# README.md
## 测试机器
Mac M2
- vcpu 8
- memory 16G
## 安装程序
- python 3.12
- Bio 1.7.1
- cutadapt 4.9
- vsearch v2.30.0
- FastQC v0.12.1
- trimmomatic 0.39
## 运行
依次执行

In [1]:
import subprocess
import csv
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import os
import json
from collections import Counter

In [2]:
def strict_QC(input_r1, input_r2, output_dir, primer_f="", primer_r=""):
    """
    执行严格的NGS数据质控流程
    :param input_r1: Read1输入文件路径
    :param input_r2: Read2输入文件路径
    :param output_dir: 输出目录
    :param primer_f: 正向引物序列（可选）
    :param primer_r: 反向引物序列（可选）
    """
    # 定义接头序列
    adapter_r1 = "AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC"  # P7 adapter for read1
    adapter_r2 = "AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT"  # P5 adapter for read2

    subprocess.run(["mkdir", "-p", f"{output_dir}/fastqc_raw"])
    subprocess.run(["mkdir", "-p", f"{output_dir}/fastqc_trimmed"])
    
    # FastQC原始数据质控
    subprocess.run(["fastqc", "-t", "14", input_r1, input_r2, "-o", f"{output_dir}/fastqc_raw"])
    
    # cutadapt切除接头与引物
    cutadapt_cmd = [
            "cutadapt",
            "-a", adapter_r2,  # R2的3'端接头（P5）
            "-A", adapter_r1,  # R1的3'端接头（P7）
            "-o", f"{output_dir}/F.fq.gz",
            "-p", f"{output_dir}/R.fq.gz",
            "--minimum-length", "50",
            "--max-n", "0",
            "--error-rate", "0.1",
            f"--json={output_dir}/cutadapt.json",
            "--cores=14"
        ]
        
    # 添加引物切除参数
    if primer_f and primer_r:
        cutadapt_cmd.extend(["-g", f"^{primer_f}", "-G", f"^{primer_r}"])
    
    cutadapt_cmd.extend([input_r1, input_r2])
    
    subprocess.run(cutadapt_cmd)
    
    # 切除后质控验证
    try:
        with open(f"{output_dir}/cutadapt.json") as f:
            log_data = json.load(f)
            
        total_pairs = log_data["read_counts"]["input"]
        kept_pairs = log_data["read_counts"]["output"]
        kept_ratio = kept_pairs / total_pairs * 100
        
        print(f"原始序列对: {total_pairs}")
        print(f"保留序列对: {kept_pairs} ({kept_ratio:.2f}%)")
        
        # 验证标准
        if kept_ratio < 90:
            print("\n⚠️ 警告: 保留率低于90%，建议检查接头/引物设计")
        else:
            print("\n✅ 保留率符合质控标准(>90%)")
                
    except json.JSONDecodeError as e:
        print(f"JSON解析失败: {e}")
    except FileNotFoundError:
        print("❌ cutadapt.json文件未生成，请检查命令执行")
    except KeyError as e:
        print(f"❌ JSON结构异常，缺失关键字段: {e}")
    
    # 额外质控：切除后FastQC验证
    subprocess.run([
        "fastqc", 
        "-t", "14",
        f"{output_dir}/F.fq.gz", 
        f"{output_dir}/R.fq.gz",
        "-o", f"{output_dir}/fastqc_trimmed"
    ])

In [3]:
def merget(input_forward,input_reverse,output_merged):
    vsearch_command = f"vsearch --fastq_mergepairs {input_forward} --reverse {input_reverse} --fastqout {output_merged} --fastq_allowmergestagger"
    subprocess.run(vsearch_command, shell=True)

In [4]:
def QC_merger(input_fastq,output_dir,output_fastq):
    subprocess.run(["mkdir", "-p", output_dir])
    subprocess.run(["fastqc", "-t", "14", input_fastq, "-o", output_dir])
    subprocess.run(["trimmomatic", "SE", "-threads", "14", "-phred33", input_fastq, output_fastq, "LEADING:3", "TRAILING:3", "SLIDINGWINDOW:4:15"])

In [5]:
class BarcodeClassifier:
    def __init__(self, input_fastq, csv_file, output_directory):
        self.input_fastq = input_fastq
        self.csv_file = csv_file
        self.output_directory = output_directory
        self.barcode_pairs = self.read_barcodes_from_csv()
        self.barcode_handles = {}

    def read_barcodes_from_csv(self):

        barcode_pairs = []
        with open(self.csv_file, 'r') as csvfile:
            csvreader = csv.reader(csvfile)
            next(csvreader)  # 跳过标题
            for row in csvreader:
                barcode1, barcode2 = row[0].strip(), row[1].strip()
                barcode_pairs.append((barcode1, barcode2))
        return barcode_pairs

    def correct_sequence(self, sequence):
        
        base_F = "ATCG"
        base_R = "TAGC"
        complement = {f: r for f, r in zip(base_F, base_R)}
        return ''.join(complement.get(base, base) for base in reversed(sequence))

    def classify_by_barcodes(self):
        unmatched_handle = open(os.path.join(self.output_directory, "unmatched_output.fastq"), "w")

        with open(self.input_fastq, "r") as handle:
            for record in SeqIO.parse(handle, "fastq"):
                found_match = False
                for barcode1, barcode2 in self.barcode_pairs:
                    if str(record.seq).startswith(barcode1) and str(record.seq).endswith(barcode2):
                        barcode_pair_name = f"{barcode1}_{barcode2}"
                        if barcode_pair_name not in self.barcode_handles:
                            self.barcode_handles[barcode_pair_name] = open(os.path.join(self.output_directory, f"{barcode_pair_name}_output.fastq"), "w")
                        SeqIO.write(record, self.barcode_handles[barcode_pair_name], "fastq")
                        found_match = True
                        break

                if not found_match:
                    corrected_seq = self.correct_sequence(str(record.seq))
                    for barcode1, barcode2 in self.barcode_pairs:
                        if corrected_seq.startswith(barcode1) and corrected_seq.endswith(barcode2):
                            barcode_pair_name = f"{barcode1}_{barcode2}"
                            if barcode_pair_name not in self.barcode_handles:
                                self.barcode_handles[barcode_pair_name] = open(os.path.join(self.output_directory, f"{barcode_pair_name}_output.fastq"), "w")

                            # 创建一个包含修正序列的新SeqRecord
                            corrected_record = SeqRecord(Seq(corrected_seq), id=record.id, description=record.description, letter_annotations={"phred_quality": record.letter_annotations['phred_quality']})
                            SeqIO.write(corrected_record, self.barcode_handles[barcode_pair_name], "fastq")
                            found_match = True
                            break

                if not found_match:
                    SeqIO.write(record, unmatched_handle, "fastq")

        for handle in self.barcode_handles.values():
            handle.close()
        unmatched_handle.close()


In [6]:
def process_sequences(config_file):
    with open(config_file, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)  # 跳过标题
        for row in csvreader:
            barcode1 = row[0].strip()
            barcode2 = row[1].strip()
            template = row[2].strip().upper()
            spacer = row[3].strip().upper()
            start_base = int(row[4].strip())
            end_base = int(row[5].strip())
            ind = template.index(spacer)
            sequences = []
            file_path = file + "/" + barcode1 + "_" + barcode2 + "_output.fastq"
            with open(file_path, "r") as seq_file:
                line_number = 0
                for line in seq_file:
                    line_number += 1
                    if line_number % 4 == 2:
                        sequence = line.strip()
                        extracted_sequence = sequence[ind+start_base:ind+end_base]
                        sequences.append(extracted_sequence)
    
            sequence_counts = Counter(sequences)
    
            output_file_path = file_path.replace('.fastq', '_counts.csv')
    
            with open(output_file_path, 'w', newline='') as output_csvfile:
                writer = csv.writer(output_csvfile)
                writer.writerow(['Extracted Sequence', 'Count'])  
                for seq, count in sequence_counts.items():
                    writer.writerow([seq, count])


In [7]:
def process_base_counts(config_file, output_file):
    results = []
    with open(config_file, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)  # 跳过标题
        for row in csvreader:
            barcode1 = row[0].strip()
            barcode2 = row[1].strip()
            template = row[2].strip().upper()
            spacer = row[3].strip().upper()
            base_windows = int(row[6].strip())
            file_path = file_1 + "/" + barcode1 + "_" + barcode2 + "_output.fastq"
            sequences = [] 
            with open(file_path, 'r') as file:
                line_number = 0

    
                for line in file:
                    line_number += 1
                    if line_number % 4 == 2:
                        sequences.append(line.strip())

            ind = template.index(spacer)
            base_windows_ind = ind + base_windows - 1

            num_A = 0
            num_T = 0
            num_C = 0
            num_G = 0

            for seq in sequences:
                if len(seq) > base_windows_ind:
                    if seq[base_windows_ind] == 'A':
                        num_A += 1
                    elif seq[base_windows_ind] == 'T':
                        num_T += 1
                    elif seq[base_windows_ind] == 'C':
                        num_C += 1
                    elif seq[base_windows_ind] == 'G':
                        num_G += 1

            results.append([file_path, spacer, base_windows, num_A, num_T, num_C, num_G])

    # 写入结果到CSV文件
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([ 'Spacer', 'Base Windows', 'A', 'T', 'C', 'G'])
        writer.writerows(results)


In [8]:
def NGS(file,purpose):
    input_forward  = file + "/F.fq.gz"
    input_reverse = file + "/R.fq.gz"
    output_merged = file + "/merged.fasta"
    merget(input_forward,input_reverse,output_merged)
    input_fastq = output_merged
    output_dir = file + "/fastqc_output"
    output_fastq = file + "/output_trimmed.fastq"
    QC_merger(input_fastq,output_dir,output_fastq)
    input_fastq = output_fastq
    tempalte_CSV = file + "/tempalte_CSV.csv"
    output_directory = file 
    classifier = BarcodeClassifier(input_fastq, tempalte_CSV, output_directory)
    classifier.classify_by_barcodes()
    if purpose == 1:
        config_file = tempalte_CSV
        process_sequences(config_file)
    elif purpose == 2:
        config_file = tempalte_CSV
        output_file = file+ "/results.csv"
        process_base_counts(config_file, output_file)


上述程序直接点击运行即可，不要进行任何修改，如果想修改的话请复制后进行修改
二代测序数据下载后请自行解压，并且把文件名修改为F.fq和R.fq
上传后确定文件夹后，在file中输入上传的路径，注意所有的后续文件都会有在这个文件夹中生成，建议每次都新建一个文件夹
purpose只有两个选项有意义，1或者2，1是针对spacer定位的区域进行扫描，2是针对碱基编辑器


In [9]:
strict_QC('./data3/X-LHF0186_L2_1.fq.gz', './data3/X-LHF0186_L2_2.fq.gz', './data3', primer_f="", primer_r="")

application/gzip
application/gzip


Started analysis of X-LHF0186_L2_1.fq.gz
Started analysis of X-LHF0186_L2_2.fq.gz
Approx 5% complete for X-LHF0186_L2_1.fq.gz
Approx 5% complete for X-LHF0186_L2_2.fq.gz
Approx 10% complete for X-LHF0186_L2_1.fq.gz
Approx 10% complete for X-LHF0186_L2_2.fq.gz
Approx 15% complete for X-LHF0186_L2_1.fq.gz
Approx 15% complete for X-LHF0186_L2_2.fq.gz
Approx 20% complete for X-LHF0186_L2_1.fq.gz
Approx 20% complete for X-LHF0186_L2_2.fq.gz
Approx 25% complete for X-LHF0186_L2_1.fq.gz
Approx 25% complete for X-LHF0186_L2_2.fq.gz
Approx 30% complete for X-LHF0186_L2_1.fq.gz
Approx 30% complete for X-LHF0186_L2_2.fq.gz
Approx 35% complete for X-LHF0186_L2_1.fq.gz
Approx 35% complete for X-LHF0186_L2_2.fq.gz
Approx 40% complete for X-LHF0186_L2_1.fq.gz
Approx 40% complete for X-LHF0186_L2_2.fq.gz
Approx 45% complete for X-LHF0186_L2_1.fq.gz
Approx 45% complete for X-LHF0186_L2_2.fq.gz
Approx 50% complete for X-LHF0186_L2_1.fq.gz
Approx 50% complete for X-LHF0186_L2_2.fq.gz
Approx 55% complete 

Analysis complete for X-LHF0186_L2_1.fq.gz
Analysis complete for X-LHF0186_L2_2.fq.gz
This is cutadapt 4.9 with Python 3.12.11
Command line parameters: -a AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -A AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC -o ./data3/F.fq.gz -p ./data3/R.fq.gz --minimum-length 50 --max-n 0 --error-rate 0.1 --json=./data3/cutadapt.json --cores=14 ./data3/X-LHF0186_L2_1.fq.gz ./data3/X-LHF0186_L2_2.fq.gz
Processing paired-end reads on 14 cores ...
Finished in 152.301 s (10.934 µs/read; 5.49 M reads/minute).

=== Summary ===

Total read pairs processed:         13,929,232
  Read 1 with adapter:              10,710,959 (76.9%)
  Read 2 with adapter:              10,675,003 (76.6%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs with too many N:                  16,616 (0.1%)
Pairs written (passing filters):    13,912,616 (99.9%)

Total basepairs processed: 4,178,769,600 bp
  Read 1: 2,089,384,800 bp
  Read 2: 2,089,384,800 bp
Total written (filter

Started analysis of F.fq.gz
Started analysis of R.fq.gz
Approx 5% complete for F.fq.gz
Approx 5% complete for R.fq.gz
Approx 10% complete for F.fq.gz
Approx 10% complete for R.fq.gz
Approx 15% complete for F.fq.gz
Approx 15% complete for R.fq.gz
Approx 20% complete for F.fq.gz
Approx 20% complete for R.fq.gz
Approx 25% complete for F.fq.gz
Approx 25% complete for R.fq.gz
Approx 30% complete for F.fq.gz
Approx 30% complete for R.fq.gz
Approx 35% complete for F.fq.gz
Approx 35% complete for R.fq.gz
Approx 40% complete for F.fq.gz
Approx 40% complete for R.fq.gz
Approx 45% complete for F.fq.gz
Approx 45% complete for R.fq.gz
Approx 50% complete for F.fq.gz
Approx 50% complete for R.fq.gz
Approx 55% complete for F.fq.gz
Approx 55% complete for R.fq.gz
Approx 60% complete for F.fq.gz
Approx 60% complete for R.fq.gz
Approx 65% complete for F.fq.gz
Approx 65% complete for R.fq.gz
Approx 70% complete for F.fq.gz
Approx 70% complete for R.fq.gz
Approx 75% complete for F.fq.gz
Approx 75% complet

Analysis complete for F.fq.gz
Analysis complete for R.fq.gz


In [10]:
file = "./data3"

In [11]:
purpose = 1

In [12]:
NGS(file,purpose)

vsearch v2.30.0_macos_aarch64, 16.0GB RAM, 8 cores
https://github.com/torognes/vsearch

Merging reads 100%
  13912616  Pairs
  13853442  Merged (99.6%)
     59174  Not merged (0.4%)

Pairs that failed merging due to various reasons:
     25546  too few kmers found on same diagonal
       553  multiple potential alignments
     14398  too many differences
     18658  alignment score too low, or score drop too high
        19  overlap too short

Statistics of all reads:
    142.76  Mean read length

Statistics of merged reads:
    148.66  Mean fragment length
     26.57  Standard deviation of fragment length
      0.12  Mean expected error in forward sequences
      0.12  Mean expected error in reverse sequences
      0.03  Mean expected error in merged sequences
      0.05  Mean observed errors in merged region of forward sequences
      0.04  Mean observed errors in merged region of reverse sequences
      0.08  Mean observed errors in merged region


biosequence/fasta


Started analysis of merged.fasta
Approx 5% complete for merged.fasta
Approx 10% complete for merged.fasta
Approx 15% complete for merged.fasta
Approx 20% complete for merged.fasta
Approx 25% complete for merged.fasta
Approx 30% complete for merged.fasta
Approx 35% complete for merged.fasta
Approx 40% complete for merged.fasta
Approx 45% complete for merged.fasta
Approx 50% complete for merged.fasta
Approx 55% complete for merged.fasta
Approx 60% complete for merged.fasta
Approx 65% complete for merged.fasta
Approx 70% complete for merged.fasta
Approx 75% complete for merged.fasta
Approx 80% complete for merged.fasta
Approx 85% complete for merged.fasta
Approx 90% complete for merged.fasta
Approx 95% complete for merged.fasta


Analysis complete for merged.fasta


TrimmomaticSE: Started with arguments:
 -threads 14 -phred33 ./data3/merged.fasta ./data3/output_trimmed.fastq LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15
Input Reads: 13853442 Surviving: 13853283 (100.00%) Dropped: 159 (0.00%)
TrimmomaticSE: Completed successfully
